In [ ]:
import io, os, sys, types
from IPython import get_ipython
from Utils import NotebookFinder
from model import create_model
import datetime
from config_file import config
from keras import callbacks
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.compat.v1 import GPUOptions,Session
from tensorflow.compat.v1 import InteractiveSession

In [ ]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
os.makedirs(log_dir)
Batch_size = config["model"]["BATCH_SIZE"]
FEATURES_SIZE = config["model"]["FEATURES_SIZE"]

In [ ]:
def train_model(features_train,lables_train):
   # features_train = tf.reshape(features_train, [-1, FEATURES_SIZE,FEATURES_SIZE, 1])
    os.environ['CUDA_VISIBLE_DEVICES'] = '/gpu:0'
    model = create_model()
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
   # Creating Keras callbacks 
    print("kkkkkkkk")
   
    tensorboard_callback = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    model_checkpoint_callback = callbacks.ModelCheckpoint(
        'training_checkpoints/weights.{epoch:02d}-{val_loss:.2f}.hdf5', period=5,monitor='val_acc',verbose=1, 
                            save_best_only=True)
    os.makedirs('training_checkpoints/', exist_ok=True)
    early_stopping_checkpoint = callbacks.EarlyStopping(patience=5)

 
    history = model.fit(
              features_train,
              lables_train,
              batch_size = Batch_size,
              epochs=3, 
              #shuffle = True,
              validation_split = 0.2,
              callbacks=[tensorboard_callback,
                         model_checkpoint_callback,
                         early_stopping_checkpoint]
    )
    
    
    return history